In [3]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException


from urllib.parse import quote
from pyquery import PyQuery as pq
from pymongo import MongoClient

In [20]:
browser = webdriver.Chrome()
browser.get('https://www.baidu.com')

In [23]:
wait = WebDriverWait(browser, 10)
browser.get('https://www.jd.com/')

In [54]:
def get_page_source():
    try:
        # 让滚动条滚到最下面获取所有的li
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        time.sleep(3)
        # 等待所有的li加载完成
        wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#J_goodsList ul'))
        )
        return browser.page_source
    
    except TimeoutException as e:
        print('Time out')
        get_page_source(page)
    except Exception as e:
        print("Error is {}".format(e))
        
def parse_url(page_source):
    """解析网页，返回相关商品信息"""
    print('开始解析')
    doc = pq(page_source)
    items = doc('#J_goodsList ul li').items()
    product_list = []
    items = doc('#J_goodsList .gl-warp .gl-item').items()
    for item in items:
        product_data = {}
        product_data['title'] = item('.p-name a').attr('title')  # 获取商品描述
        product_data['price'] = item('.p-price strong').text()
        product_data['comment_count'] = item('.p-commit strong ').text()
        product_data['shop'] = item('.p-shop .J_im_icon a').text()
        product_data['href'] = item('.p-name a').attr('href')
        print(product_data)
        yield product_data

def save_to_mongo(data):
    for product_data in data:
        try:
            collection.insert_one(product_data)
        except Exception as e:
            print('error is {}'.format(e))
    print('保存成功')

def main(page):
    print('正在获取第{}页'.format(page))
    # 1.执行js，让滚动条滚到最下面，获取所有的li后，返回网页源代码
    page_source = get_page_source()
    # 2.对网页源代码进行解析
    product_list = parse_url(page_source)
    # 3.保存到数据库
    save_to_mongo(data)
    # 4.翻页，继续1，2，3
    page_input = browser.find_element_by_css_selector('#J_bottomPage .input-txt')
    submit_button = browser.find_element_by_css_selector('#J_bottomPage .btn')
    page_input.clear()
    page_input.send_keys(page)
    print('发送成功 {}'.format(page))
    submit_button.click()
    
if __name__ == "__main__":
    client = MongoClient(host='localhost', port=27017)
    collection = client['jd']['jd_product']
    browser.get('https://www.jd.com')
    keyword = "手机"
    input_ = browser.find_element_by_css_selector('#key')
    input_.send_keys(keyword)
    input_.send_keys(Keys.ENTER)
    time.sleep(1)

    _ = main(2)

正在获取第2页
保存成功
发送成功 2


In [53]:
items = doc('#J_goodsList .gl-warp .gl-item').items()
# for item in items:
#     product_data = {}
#     product_data['title'] = item('.p-name a').attr('title')  # 获取商品描述
#     product_data['price'] = item('.p-price strong').text()
#     product_data['comment_count'] = item('.p-commit strong ').text()
#     product_data['shop'] = item('.p-shop .J_im_icon a').text()
#     product_data['href'] = item('.p-name a').attr('href')
#     print(product_data)
# #     break
for i in _:
    print(i)

<generator object parse_url at 0x00000174E13A65E8>


In [158]:

client = MongoClient(host='localhost', port=27017)
collection = client['jd']['jd_product']
# print(collection)

collection = client['douban']['tv']
print(collection.count_documents({}))

864
